https://studymachinelearning.com/keras-fit-fit_generator-train_on_batch/

In [19]:
import tensorflow as tf
import os
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
import math

In [20]:
from PIL import Image
import numpy as np

In [21]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
 
    def __init__(self, path, N, y, batch_size, dim, n_classes):
        
        self.path= path
        self.N= N 
        
        self.y= y
        
        self.batch_size = batch_size

        self.n_classes = n_classes
        self.dim = dim
        self.n = 0
        self.list_IDs = np.arange(self.N)
        self.on_epoch_end()
        
    def __next__(self):
        data = self.__getitem__(self.n)
        self.n += 1
        
        if self.n >= self.__len__():
#         if self.n >= int( self.N/self.batch_size ):
   
            self.indexes = np.arange( self.N )
            np.random.shuffle(self.indexes)
#             self.on_epoch_end
            self.n = 0
        
        return data
    
    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        batch_ids = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(batch_ids)
        y = self._generate_y(batch_ids)
        return X, y
        
    def on_epoch_end(self):
    
        self.indexes = np.arange( self.N )
        np.random.shuffle(self.indexes)
        
    def _generate_x(self, batch_ids):
        X = np.empty((self.batch_size, *self.dim))
        
        for i, ID in enumerate(batch_ids):
            filename= self.path + str(ID) + '.png'
            img = Image.open( filename )
            X[i,] = np.asarray( img, dtype="int16" )
        
            X = (X/255).astype('float32')
        return X[:,:,:, np.newaxis]
    
    def _generate_y(self, batch_ids):
        
        y = np.empty(self.batch_size)
        
        for i, ID in enumerate(batch_ids):
            y[i] = self.y[ID]

        return keras.utils.to_categorical(
                y,num_classes=self.n_classes)

In [22]:
n_classes = 10
input_shape = (28, 28)

In [23]:
y = np.loadtxt("data/labels.txt", dtype='int16')

path= 'data/'
train_generator = DataGenerator(path, 2000, y, batch_size = 64,
                                dim = input_shape,
                                n_classes=10)
val_generator =  DataGenerator(path, 2000, y, batch_size=64, 
                               dim = input_shape, 
                               n_classes= n_classes)

In [24]:
images, labels = next(train_generator)
print(images.shape)
print(labels.shape)

(64, 28, 28, 1)
(64, 10)


In [25]:
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28 , 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [27]:
model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=2,
        validation_data=val_generator,
        validation_steps=validation_steps)

Epoch 1/2
32/32 [==============================] - 12s 364ms/step - loss: 2.3032 - accuracy: 0.1069 - val_loss: 2.3028 - val_accuracy: 0.1108
Epoch 2/2
32/32 [==============================] - 12s 371ms/step - loss: 2.3025 - accuracy: 0.1143 - val_loss: 2.3024 - val_accuracy: 0.1133
